In [163]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from selenium.webdriver.common.keys import Keys

In [164]:
browser = webdriver.Chrome('/Users/joseph/Desktop/driver/chromedriver')
browser.get('https://www.linkedin.com/uas/login')
file = open('/Users/joseph/Desktop/driver/config.txt')
lines = file.readlines()
username = lines[0].rstrip()
password = lines[1].rstrip()

elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

In [165]:
link = 'https://www.linkedin.com/search/results/all/?keywords=new%20grad&origin=GLOBAL_SEARCH_HEADER'
browser.get(link)

In [166]:
all_links = []
unique_id = set()

for x in range(100):

    for i in range(5):
        # Scroll page down
        browser.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    array_links = [a['href'] for a in soup.find_all('a', href=True)]
    
    for list in array_links:
        if (list.find('/jobs/view/') != -1):
            result = re.search('/jobs/view/(.*)/',list)
            if (result.group(1) not in unique_id):
                unique_id.add(result.group(1))
                all_links.append(list)
                
    element = browser.find_elements_by_css_selector("[aria-label=Next]")
    element[0].click()

In [167]:
for index, link in enumerate(all_links):
    all_links[index] = 'https://www.linkedin.com' + all_links[index]

In [168]:
import json
with open('info.json', 'w') as f:
    link = {
        'link' : all_links
    }
    json.dump(link,f)

In [169]:
len(all_links)

990

In [170]:
import json
with open('info.json', 'r') as f:
    p = json.load(f)

In [171]:
all_links = p.get("link","")

In [172]:
class jobPostInfo:
    def __init__(instance, job_title, location, company, body_text):
        instance.job_title = job_title
        instance.location = location
        instance.company = company
        instance.body_text = body_text

In [187]:
company_text = []
for list in all_links:
    l = list
    browser.get(l)

    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    location = ""
    job_title = str(soup.find_all("h1", {"class" : "jobs-top-card__job-title t-24"}, text = True)[0].text)
    locationList = soup.find_all("span", {"class" : "jobs-top-card__bullet"},text = True)
    if len(locationList) > 0:
        location = str(soup.find_all("span", {"class" : "jobs-top-card__bullet"}, text = True)[0].text)
    elif len(locationList) == 0:
        location = str(soup.find_all("a", {"class" : "jobs-top-card__exact-location t-black--light link-without-visited-state"}, text = True)[0].text)
    else:
        location = "not found"
        
    company = soup.find_all("a", {"data-control-name" : "company_link"}, text = True)
    if len(company) > 0:
        company = str(soup.find_all("a", {"data-control-name" : "company_link"}, text = True)[0].text)
    elif len(company) == 0:
        company = soup.find_all("span", {"class" : "a11y-text"})[0].nextSibling.strip()
    else:
        company = "location not found"
        
    test_stuff = soup.find('article').find('span')
    location = location.strip('\n').strip()
    company = company.strip('\n').strip()
    jobclass = jobPostInfo(job_title, location, company, test_stuff)
    company_text.append(jobclass)

IndexError: list index out of range

In [188]:
len(company_text)

342

In [175]:
for companies in company_text:
    companies.body_text = companies.body_text.text

In [176]:
file_arry = []
with open('company_info.json', 'w') as f:
    for ob in company_text:
        file_arry.append(ob.__dict__ )
    json.dump(file_arry,f)

In [184]:
all_links.remove(all_links[520])